In [ ]:
import gym
import gym_anytrading
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import A2C

env = gym.make('stocks-v0', df=fu_df, frame_bound=(5,120), window_size=5 )
env.action_space

In [ ]:
state = env.reset()
while True:
    action = env.action_space.sample()
    n_state, rewards, done, info = env.step(action)
    if done:
        print('info', info)
        break

plt.figure(figsize=(15,6))
plt.cla()
env.render_all()
plt.show()

In [ ]:
from gym_anytrading.envs import StocksEnv
from finta import TA

#fu_df['Volume'] = fu_df['Volume'].apply(lambda x: float(x.replace(".", "")))
fu_df['SMA'] = TA.SMA(fu_df, 12)
fu_df['RSI'] = TA.RSI(fu_df)
fu_df['OBV'] = TA.OBV(fu_df)
fu_df.fillna(0,inplace=True)
fu_df.head(15)

In [ ]:
def add_signals(env):
    start = env.frame_bound[0] - env.window_size
    end = env.frame_bound[1]
    prices = env.df.loc[:, 'Low'].to_numpy()[start:end]
    signal_features = env.df.loc[:, ['Low', 'Volume', 'SMA', 'RSI', 'OBV']].to_numpy()[start:end]
    return prices, signal_features

class MyCustomEnv(StocksEnv):
    _process_data = add_signals

env2 = MyCustomEnv(df=fu_df, window_size=12, frame_bound=(12,50))
env_maker = lambda: env2
env = DummyVecEnv([env_maker])

In [ ]:
from stable_baselines3.a2c.policies import MlpPolicy

model = A2C(MlpPolicy, env, verbose=1, device='cuda')
model.learn(total_timesteps=1000000)



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

env = MyCustomEnv(df=fu_df, window_size=12, frame_bound=(60,120))
state = env.reset()
while True:
    n_state = n_state[np.newaxis, ...]
    action = env.action_space.sample()
    n_state, rewards, done, info = env.step(action)
    if done:
        print('info', info)
        break

plt.figure(figsize=(15,6))
plt.cla()
env.render_all()
plt.show()